**Modelo tipo 1 e 2**

* Tipo 1: demanda <= oferta

* Tipo 2: demanda >= oferta

* fim no cliente

In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [2]:
df_ori_dest = pd.read_csv('../dados/origem_porto.csv')
df_ori_trans = pd.read_csv('../dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('../dados/transbordo_porto.csv')

In [3]:
df_supply = pd.read_csv('../dados/supply.csv')
df_cap_port = pd.read_csv('../dados/cap_porto.csv')
df_cap_trans = pd.read_csv('../dados/cap_transbordo.csv')
df_demand = pd.read_csv('../dados/demand.csv')

In [4]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_cap_trans.shape[0]
qnt_port = df_cap_port.shape[0]
qnt_cli = df_demand.shape[0]

In [5]:
df_supply

,0
0,543
1,856
2,622
3,797
4,844
...,...
92,583
93,515
94,645
95,107


In [6]:
df_demand

,0
0,12141
1,3472
2,30864
3,42111
4,39395
...,...
66,18358
67,37741
68,15654
69,5976


In [7]:
df_ori_dest

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,76.185710,86.540922,68.321686,89.283584,24.900956,117.443597,92.582642,31.363673,84.422101,82.787902,...,71.216537,23.880235,77.439008,70.662058,67.837924,79.106854,81.772367,68.335174,49.501838,48.644737
1,78.514855,91.085542,71.577963,92.190273,29.372096,117.211986,97.186929,35.957664,88.426747,84.794717,...,75.287779,28.404845,78.409959,73.650244,72.236766,81.086068,82.941290,70.553469,53.983583,52.553363
2,74.529951,81.610980,65.456233,86.831662,19.690404,118.695523,87.510493,26.361184,80.421927,81.515564,...,67.133872,19.105099,77.440661,68.158122,63.205671,77.882537,81.523101,66.856264,44.734237,44.893616
3,78.306883,93.632268,72.298227,92.507124,32.844068,114.969547,99.928294,38.725289,90.015181,84.285989,...,76.988103,31.015661,77.070357,74.086220,74.472051,80.561231,81.744969,70.266237,56.415601,54.088757
4,74.898117,81.003783,65.547882,87.014114,18.676873,119.556728,86.822817,25.745586,80.179798,81.959522,...,66.870430,18.635021,78.121464,68.337984,62.740405,78.337935,82.159182,67.266443,44.220538,44.755692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,83.147368,97.155315,77.027829,97.322622,34.749849,118.907884,103.200496,41.987122,94.370843,89.099315,...,81.270236,34.480278,81.645321,78.880649,78.327477,85.372858,86.377627,75.103427,60.072490,58.465716
93,71.606570,95.080665,68.070552,86.992487,39.468144,104.528503,102.106078,42.030084,88.162178,76.844321,...,75.636013,34.140416,67.987764,69.059603,74.964909,73.110196,72.902785,63.457702,57.930165,52.632948
94,76.883288,89.967390,69.993188,90.573285,28.908241,115.863202,96.163195,34.935003,86.991310,83.169995,...,73.877737,27.296300,76.864806,72.039989,70.989892,79.462192,81.375220,68.922867,52.808969,51.098149
95,78.597140,93.984441,72.631541,92.823307,33.132365,115.089836,100.274950,39.069169,90.369836,84.557843,...,77.344238,31.364081,77.281822,74.408430,74.828871,80.832310,81.967799,70.552562,56.769569,54.443277


In [8]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]
O = [i + qnt_orig + qnt_trans + qnt_port for i in range(qnt_cli)]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in O:
    demand[i] = df_demand['0'][i - O[0]]

cap_transbordo = {}
for i in K:
    cap_transbordo[i] = df_cap_trans['0'][i - K[0]]

cap_port = {}
for i in M:
    cap_port[i] = df_cap_port['0'][i - M[0]]

cost = {}
for i in N:
    for j in M:
        cost[i, j] = df_ori_dest[str(j - M[0])][i]
    for k in K:
        cost[i, k] = df_ori_trans[str(k - K[0])][i]

for j in M:
    for k in K:
        cost[k, j] = df_trans_porto[str(j - M[0])][k - K[0]]
    for o in O:
        cost[j, o] = 0


In [9]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]

print(oferta_total, demanda_total)

54973 1854563


In [10]:
m = gp.Model("probrema")

In [11]:
X = {}
for i in N:
    for j in M:
        X[i, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_port_{}".format(i, j))

for i in N:
    for k in K:
        X[i, k] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_tranship_{}".format(i, k))
                
for j in M:
    for k in K:
        X[k, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="tranship_{}_port_{}".format(k, j))

for j in M:
    for o in O:
        X[j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(j, o))

In [12]:
m.setObjective(
    gp.quicksum((X[i, j] * cost[i, j]) for i in N for j in M) + \
    gp.quicksum((X[i, k] * cost[i, k]) for i in N for k in K) + \
    gp.quicksum((X[k, j] * cost[k, j]) for j in M for k in K),
    sense=gp.GRB.MINIMIZE
)

In [13]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) == supply[i]
        )
else:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) <= supply[i]
        )

In [14]:
if oferta_total <= demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) == demand[o]
        )

In [15]:
for j in M:
    m.addConstr(
        (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K)) <= cap_port[j]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) <= cap_transbordo[k]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) == gp.quicksum(X[k, j] for j in M)
    )

for j in M:
    m.addConstr(
        gp.quicksum(X[j, o] for o in O) == (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K))
    )

In [16]:
rest = m.addConstrs(
    gp.quicksum(X[i, j] for i in N) >= 0 for j in M
)

rest = m.addConstrs(
    gp.quicksum(X[i, k] for i in N) >= 0 for k in K
)

rest = m.addConstrs(
    gp.quicksum(X[k, j] for k in K) >= 0 for j in M
)

In [17]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 425 rows, 11166 columns and 38842 nonzeros
Model fingerprint: 0x6da9e401
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+05]
Presolve removed 113 rows and 0 columns
Presolve time: 0.02s
Presolved: 312 rows, 11166 columns, 30587 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.949e+04
 Factor NZ  : 3.281e+04 (roughly 5 MB of memory)
 Factor Ops : 4.852e+06 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.04 seconds (0.01 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with d

In [18]:
m.write("out.sol")